In [ ]:
# !pip install tensorflow==2.6


In [ ]:
# !pip install stable_baselines

In [15]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle
import tensorflow
import time


In [ ]:
!pip install tensorflow==1.14.0

In [ ]:
!pip install stable-baselines[mpi]==2.10.0


In [16]:
# from stable_baselines import A2C

from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines.common.vec_env import DummyVecEnv


In [17]:

# shares normalization factor
# 100 shares per trade
HMAX_NORMALIZE = 100
# initial amount of money we have in our account
INITIAL_ACCOUNT_BALANCE=100000000
# total number of stocks in our portfolio
STOCK_DIM = 236
# transaction fee: 1/1000 percentage
TRANSACTION_FEE_PERCENT = 0.001
REWARD_SCALING = 1e-4

class StockEnvTrain(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df,day = 0):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.df = df

        # action_space normalization and shape is STOCK_DIM
        self.action_space = spaces.Box(low = -1, high = 1,shape = (STOCK_DIM,)) 
        # Shape = 709: [Current Balance]+[prices 1-236]+[owned shares 1-236] 
        # +[feature 1-236]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (709,))
        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.terminal = False             
        # initalize state
        self.state = [INITIAL_ACCOUNT_BALANCE] + \
                      self.data.close.values.tolist() + \
                      [0]*STOCK_DIM + \
                      self.data.percent_good_subfund.values.tolist()
                      
        # initialize reward
        self.reward = 0
        self.cost = 0
        # memorize all the total balance change
        self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
        self.rewards_memory = []
        self.trades = 0
        #self.reset()
        self._seed()


    def _sell_stock(self, index, action):
        # perform sell action based on the sign of the action
        if self.state[index+STOCK_DIM+1] > 0:
            #update balance
            self.state[0] += \
            self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
             (1- TRANSACTION_FEE_PERCENT)

            self.state[index+STOCK_DIM+1] -= min(abs(action), self.state[index+STOCK_DIM+1])
            self.cost +=self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
             TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            pass

    
    def _buy_stock(self, index, action):
        # perform buy action based on the sign of the action
        available_amount = self.state[0] // self.state[index+1]
        # print('available_amount:{}'.format(available_amount))

        #update balance
        self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                          (1+ TRANSACTION_FEE_PERCENT)

        self.state[index+STOCK_DIM+1] += min(available_amount, action)

        self.cost+=self.state[index+1]*min(available_amount, action)* \
                          TRANSACTION_FEE_PERCENT
        self.trades+=1
        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            plt.plot(self.asset_memory,'r')
            # plt.savefig('results/account_value_train.png')
            plt.close()
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            
            #print("end_total_asset:{}".format(end_total_asset))
            df_total_value = pd.DataFrame(self.asset_memory)
            # df_total_value.to_csv('results/account_value_train.csv')
            #print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):61]))- INITIAL_ACCOUNT_BALANCE ))
            #print("total_cost: ", self.cost)
            #print("total_trades: ", self.trades)
            df_total_value.columns = ['account_value']
            df_total_value['daily_return']=df_total_value.pct_change(1)
            sharpe = (252**0.5)*df_total_value['daily_return'].mean()/ \
                  df_total_value['daily_return'].std()
            #print("Sharpe: ",sharpe)
            #print("=================================")
            df_rewards = pd.DataFrame(self.rewards_memory)
            
            return self.state, self.reward, self.terminal,{}

        else:
            # print(np.array(self.state[1:29]))

            actions = actions * HMAX_NORMALIZE
            #actions = (actions.astype(int))
            
            begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            #print("begin_total_asset:{}".format(begin_total_asset))
            
            argsort_actions = np.argsort(actions)
            
            sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print('take sell action'.format(actions[index]))
                self._sell_stock(index, actions[index])

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                self._buy_stock(index, actions[index])

            self.day += 1
            self.data = self.df.loc[self.day,:]         
            #load next state
            print("stock_shares:{}".format(self.state[29:]))
            self.state =  [self.state[0]] + \
                    self.data.close.values.tolist() + \
                    list(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]) + \
                    self.data.percent_good_subfund.values.tolist()
                   
            
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            self.asset_memory.append(end_total_asset)
            print("end_total_asset:{}".format(end_total_asset))
            
            self.reward = end_total_asset - begin_total_asset            
            print("step_reward:{}".format(self.reward))
            self.rewards_memory.append(self.reward)
            
            self.reward = self.reward*REWARD_SCALING



        return self.state, self.reward, self.terminal, {}

    def reset(self):
        self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
        self.day = 0
        self.data = self.df.loc[self.day,:]
        self.cost = 0
        self.trades = 0
        self.terminal = False 
        self.rewards_memory = []
        #initiate state
        self.state = [INITIAL_ACCOUNT_BALANCE] + \
                      self.data.close.values.tolist() + \
                      [0]*STOCK_DIM + \
                      self.data.percent_good_subfund.values.tolist()
                      
        # iteration += 1 
        return self.state
    
    def render(self, mode='human'):
        return self.state

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]

In [ ]:

def train_A2C(env_train, timesteps=10000):
    start = time.time()
    model = A2C('MlpPolicy', env_train, verbose=1)
    model.learn(total_timesteps=timesteps)
    end = time.time()
    print('Training time (A2C): ', (end-start)/60,' minutes')
    return model

In [18]:
pool_prices = pd.read_csv('/content/test_data_date_index.csv').set_index("nums")
# # pool_prices.drop(columns=["Unnamed: 0"],inplace=True)
# pool_prices[pool_prices["nums"] == 0]
# pool_prices.set_index("nums")
# pool_prices.loc[0,:]

In [19]:
merged_pools = pool_prices
merged_pools.dtypes

stock_id                 object
percent_good_subfund    float64
date                     object
close                   float64
dtype: object

In [20]:
merged_pools

,stock_id,percent_good_subfund,date,close
nums,,,,
0,001055102,2.978489,2013-09-30,30.995
0,00206R102,7.539118,2013-09-30,33.820
0,002824100,4.139970,2013-09-30,33.190
0,00287Y109,3.548546,2013-09-30,44.730
0,00507V109,0.426743,2013-09-30,16.680
...,...,...,...,...
18,98978V103,3.548546,2020-03-31,117.690
18,G1151C101,7.552870,2020-03-31,163.260
18,G29183103,2.140992,2020-03-31,77.690


In [21]:
env = StockEnvTrain(merged_pools)


In [10]:
!pip install preprocessing[preprocessors]

In [48]:
from stable_baselines import GAIL, SAC
from stable_baselines import ACER
from stable_baselines import PPO2
from stable_baselines import A2C
from stable_baselines import DDPG
from stable_baselines import TD3

from stable_baselines.ddpg.policies import DDPGPolicy
from stable_baselines.common.policies import MlpPolicy, MlpLstmPolicy, MlpLnLstmPolicy
from stable_baselines.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise, AdaptiveParamNoiseSpec
from stable_baselines.common.vec_env import DummyVecEnv

In [57]:

def train_A2C(env_train, model_name, timesteps=25000):
    """A2C model"""

    start = time.time()
    model = A2C('MlpPolicy', env_train, verbose=0)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"{model_name}")
    print('Training time (A2C): ', (end - start) / 60, ' minutes')
    return model

def train_ACER(env_train, model_name, timesteps=25000):
    start = time.time()
    model = ACER('MlpPolicy', env_train, verbose=0)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"./{model_name}")
    print('Training time (A2C): ', (end - start) / 60, ' minutes')
    return model


def train_DDPG(env_train, model_name, timesteps=10000):
    """DDPG model"""

    # add the noise objects for DDPG
    n_actions = env_train.action_space.shape[-1]
    param_noise = None
    action_noise = OrnsteinUhlenbeckActionNoise(mean=np.zeros(n_actions), sigma=float(0.5) * np.ones(n_actions))

    start = time.time()
    model = DDPG('MlpPolicy', env_train, param_noise=param_noise, action_noise=action_noise)
    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"./{model_name}")
    print('Training time (DDPG): ', (end-start)/60,' minutes')
    return model

def train_PPO(env_train, model_name, timesteps=50000):
    """PPO model"""

    start = time.time()
    model = PPO2('MlpPolicy', env_train, ent_coef = 0.005, nminibatches = 8)
    #model = PPO2('MlpPolicy', env_train, ent_coef = 0.005)

    model.learn(total_timesteps=timesteps)
    end = time.time()

    model.save(f"./{model_name}")
    print('Training time (PPO): ', (end - start) / 60, ' minutes')
    return model

def train_GAIL(env_train, model_name, timesteps=1000):
    """GAIL Model"""
    from stable_baselines.gail import ExpertDataset, generate_expert_traj
    start = time.time()
    # generate expert trajectories
    model = SAC('MlpPolicy', env_train, verbose=1)
    generate_expert_traj(model, 'expert_model_gail', n_timesteps=100, n_episodes=10)

    # Load dataset
    dataset = ExpertDataset(expert_path='expert_model_gail.npz', traj_limitation=10, verbose=1)
    model = GAIL('MlpPolicy', env_train, dataset, verbose=1)

    model.learn(total_timesteps=1000)
    end = time.time()

    model.save(f"./{model_name}")
    print('Training time (PPO): ', (end - start) / 60, ' minutes')
    return model

In [ ]:
mod = train_A2C(env, "a2cmod")

In [27]:
mod.save("./a2c")

In [32]:
from stable_baselines.common.evaluation import evaluate_policy

In [ ]:
evaluate_policy(mod, env, render=True, n_eval_episodes = 100)


In [ ]:
mod2 = train_DDPG(env, "ddpg_mod")

In [ ]:
evaluate_policy(mod2, env, render=True)


In [ ]:
mod3 = train_PPO(env, "ppo_mod")

In [ ]:
evaluate_policy(mod3, env, render=True)


In [58]:
mod4 = train_GAIL(env, "gail_mod")

stock_shares:[63.6244, 13.8, 30.19, 35.2733, 100.02, 28.5733, 37.5, 225.435, 270.62, 117.5, 11.74, 46.28, 31.75, 45.23, 23.13, 73.88, 25.74, 56.75, 37.32, 68.74, 52.15, 32.64, 83.4, 15.99, 23.97, 121.5, 23.430999999999997, 48.51, 81.72, 37.88, 41.06, 59.3, 22.5575, 39.31, 69.51, 57.4, 15.895, 14.59, 115.17, 132.87, 52.4507, 41.6353, 56.9, 81.39, 23.59, 42.64, 53.1, 64.49, 43.9793, 56.46, 62.48, 60.1562, 66.78, 98.76, 34.815, 25.55, 64.7, 86.04, 50.23, 25.135, 114.11, 18.05, 25.2625, 110.16, 16.87, 20.26, 41.63, 23.8127, 35.97, 80.89, 62.87, 25.54, 158.21, 42.75, 47.14, 51.66, 92.5, 75.85, 81.9501, 21.06, 65.25, 76.27, 38.15, 22.921, 185.18, 44.4297, 17.18, 66.31, 125.42299999999999, 51.69, 86.69, 19.86, 109.36, 58.73, 11.4, 90.2375, 35.57, 20.17, 61.1, 31.87, 99.14, 51.2, 66.42, 69.9, 50.33, 127.55, 47.61, 20.44, 32.16, 43.55, 42.06, 18.998, 67.278, 29.82, 64.7, 96.21, 47.608999999999995, 41.9433, 33.28, 40.29, 17.469, 130.25, 31.421, 59.38, 44.1728, 80.16, 36.32, 49.02, 15.56, 127.59,

AttributeError: ignored